# ABSEHRD package demo
This notebook demonstrates Automated Brewering Synthetic Electronic Health Record Data (ABSEHRD) package functionality on a toy dataset.

## Setup

### Import python and ABSEHRD modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import random_split

from preprocessor import Preprocessor
from corgan import Corgan
from realism import Realism
from privacy import Privacy

ImportError: cannot import name 'preprocessor' from 'preprocessor' (/Users/haleyhunter-zinck/Documents/workspace/synth/sehrd/src/preprocessor.py)

### Set parameters for the toy dataset and demo

In [ ]:
# toy dataset
n = 10000
count_min = 5
count_max = 19
constant_value = 'helloworld'
binary_A = 'A'
binary_B = 'B'
categorical_values = ['X','Y','Z']

# synthetic data generation and validation
n_gen = round(n/2)
outcome = 'binary01'

# sehrd objects
pre = preprocessor(missing_value=-99999)
rea = realism()
pri = privacy()
cor = corgan()

### Generate the toy dataset

In [ ]:
names = ['constant','binary01', 'binaryAB', 'categorical','count','continuous']
v_constant = np.full(shape=n, fill_value=constant_value)
v_binary01 = np.random.randint(low=0, high=2, size=n)
v_binaryAB = np.concatenate((np.full(shape=n-1, fill_value=binary_A), np.array([binary_B])))
v_categorical = np.random.choice(categorical_values, size=n)
v_count = np.random.randint(low=count_min, high=count_max+1, size=n)
v_continuous = np.random.random(size=n)
x = np.column_stack((v_constant, v_binary01, v_binaryAB, v_categorical, v_count, v_continuous))
print(x)

### Split into training and testing set

In [ ]:
n_subset = round(len(x) * 0.5)
idx_trn = np.random.choice(len(x), n_subset, replace=False)
idx_tst = np.setdiff1d(range(len(x)), idx_trn)
x_trn = x[idx_trn,:]
x_tst = x[idx_tst,:]

print('Number of training samples: '+str(len(x_trn)))
print('Number of testing samples: '+str(len(x_tst)))

## Preprocessing

### Save metadata for restoring data format after synthetic data generation

In [ ]:
m = pre.get_metadata(x=x_trn, header=names)
print('var_name, var_type, min, max, zero, one, unique, missing')
print(m)

### Encode raw data matrix in preparation for training synthetic data generator
Note that count and continuous variables have been scaled between 0 and 1 while constant, categorical, and binary have been one-hot encoded.

In [ ]:
d = pre.get_discretized_matrix(x_trn, m, names)
print('Formatted matrix:')
print(d['x'])
print('\nHeader for formatted matrix:')
print(d['header'])

## Generation

### Train CorGAN model 

In [ ]:
model = cor.train(x=d['x'], n_cpu=1, debug=False)

### Generate synthetic samples

In [ ]:
s = cor.generate(model, n_gen)
print(s)

### Use metadata to restore original formatting

In [ ]:
f = pre.restore_matrix(x=s, m=m, header=d['header'])
print('Synthetic samples:')
print(f['x'])
print('\nReal samples:')
print(x)

## Realism

### Compare univariate frequency for real and synthetic features

In [ ]:
res_uni = rea.validate_univariate(r=d['x'], s=s, header=d['header'])
corr_uni = np.corrcoef(x=res_uni['frq_r'], y=res_uni['frq_s'])[0,1]
print('Correlation between feature frequencies =',np.round(corr_uni,2))

Plot the synthetic and real feature frequencies for each feature...

In [ ]:
fontsize = 14
fig, ax1 = plt.subplots(1,1)
ax1.plot([0,1],[0,1], color="gray", linestyle='--')
ax1.scatter(res_uni['frq_r'], res_uni['frq_s'], label='Frequency')
ax1.set_xlabel('Real feature frequency', fontsize=fontsize)
ax1.set_ylabel('Synthetic feature frequency', fontsize=fontsize)
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])
ax1.tick_params(axis='x', labelsize=fontsize)
ax1.tick_params(axis='y', labelsize=fontsize)
ax1.legend(fontsize=fontsize)

### Compare predictive performance
* Real: use real dataset to train predictive model and test on a separate real dataset
* GAN-train: use synthetic dataset to train predictive model and test on a real dataset
*GAN-test: use real dataset to train predictive model and test on the synthetic dataset

In [ ]:
print('Extract outcome \'', outcome,'\' from real and synthetic datasets', sep='')
print('\nHeader:')
print(d['header'])

In [ ]:
r = d['x']

idx_outcome = np.where(d['header'] == outcome+'__'+outcome)
y_r = np.reshape(np.round(np.reshape(r[:,idx_outcome], newshape=(len(r),1))).astype(int), len(r))
y_s = np.reshape(np.round(np.reshape(s[:,idx_outcome], newshape=(len(s),1))).astype(int), len(s))
x_r = np.delete(r, idx_outcome, axis=1)
x_s = np.delete(s, idx_outcome, axis=1)

print('Real dataset dimensions:',x_r.shape)
print('Synthetic dataset dimensions:',x_s.shape)

Train model for each validation test

In [ ]:
n_epoch = 20
model_type='lr'

res_gan_real = rea.gan_train(x_r, y_r, x_r, y_r, n_epoch=n_epoch, model_type=model_type)
res_gan_train = rea.gan_train(x_s, y_s, x_r, y_r, n_epoch=n_epoch, model_type=model_type)
res_gan_test = rea.gan_test(x_s, y_s, x_r, y_r, n_epoch=n_epoch, model_type=model_type)

Plot resulting ROC curves

In [ ]:
fontsize = 14
fig, ax3 = plt.subplots(1,1)
ax3.plot(res_gan_real['roc'][0], res_gan_real['roc'][1], label="Real")
ax3.plot(res_gan_train['roc'][0], res_gan_train['roc'][1], label="GAN-train")
ax3.plot(res_gan_test['roc'][0], res_gan_test['roc'][1], label="GAN-test")
ax3.plot([0,1],[0,1], color="gray", linestyle='--')
ax3.tick_params(axis='x', labelsize=fontsize)
ax3.tick_params(axis='y', labelsize=fontsize)
ax3.legend(fontsize=fontsize)
ax3.set_xlabel('False positive rate', fontsize=fontsize)
ax3.set_ylabel('True positive rate', fontsize=fontsize)

## Privacy

## Nearest neighbors
Ensure that synthetic dataset is not a copy of the real dataset by comparing distances between pairs of real and synthetic samples
* Real-real: distance between randomly selected pairs of real samples
* Real-synthetic: distance between pairs of real and synthetic samples
* Real-probabilistic: distance between a real sample and sampled binary vector where each column is sampled from a binomial where the frequency equals that in the real training set
* Real-random: distance between a real sample and a randomly sampled binary vector

In [ ]:
n_nn_sample = 100
dist_metric = 'euclidean'
n_decimal=2

idx_r = np.random.randint(low=0, high=len(r), size=min((len(r), n_nn_sample)))
idx_s = np.random.randint(low=0, high=len(s), size=min((len(s), n_nn_sample)))
res_nn = pri.assess_memorization(r[idx_r,:], s[idx_s,:], metric=dist_metric)

print('Mean nearest neighbor distance: ')
print('  > Real-real:\t\t'+str(np.round(np.mean(res_nn['real']),n_decimal)))
print('  > Real-synthetic:\t'+str(np.round(np.mean(res_nn['synth']),n_decimal)))
print('  > Real-probabilistic: '+str(np.round(np.mean(res_nn['prob']),n_decimal)))
print('  > Real-random:\t'+str(np.round(np.mean(res_nn['rand']),n_decimal)))

Plot distributions for nearest neighbor distances

In [ ]:
fontsize = 14
fig, ax2 = plt.subplots(1,1)
ax2.hist((res_nn['real'], res_nn['synth'], 
          res_nn['prob'], res_nn['rand']),
         bins=30, 
         label = ['Real-real','Real-synthetic','Real-probabilistic','Real-random'])
ax2.set_xlabel(dist_metric.capitalize()+' distance', fontsize=fontsize)
ax2.set_ylabel('Number of samples', fontsize=fontsize)
ax2.tick_params(axis='x', labelsize=fontsize)
ax2.tick_params(axis='y', labelsize=fontsize)
ax2.legend(fontsize=fontsize)
